## Example python code for using `delta-processor-axis`

We describe the whole process from the 3d model url to medial axis retrieval using docker.  
Pypi packages `requests` and `docker` are required to run this sample project.  
Note that the model url might not be available at the moment you run the code.  
You can reference this code to build an intergrated system for medial axis problem.

In [1]:
import os
import uuid
import shutil
import warnings

import requests
import docker

In [2]:
MODEL_DOWNLOAD_URL = 'http://research.prev.kr/delta/models/horse.off'

### Make a temp directory in which input and ouput files are located

In [3]:
tmpdirname = uuid.uuid4().hex
print(tmpdirname)

os.mkdir(tmpdirname)

5ad0a4d7bb8b49558cacec1870a6aed2


### Download 3D model file from the URL

In [4]:
filename = MODEL_DOWNLOAD_URL.split('/')[-1]
filepath = os.path.join(tmpdirname, filename)
with requests.get(MODEL_DOWNLOAD_URL, stream=True) as r:
    with open(filepath, 'wb') as f:
        shutil.copyfileobj(r.raw, f)

size = os.path.getsize(filepath)
size_mb = round(size / 1024 / 1024, 1)
print(f'Size of the downloaded file: {size_mb}MB')

Size of the downloaded file: 4.6MB


### Check docker image exists

In [5]:
client = docker.from_env()
image_exists = False

for image in client.images.list():
    for tag in image.tags:
        if 'delta-processor-axis:' in tag:
            image_exists = True
            break
            
if not image_exists:
    warnings.warn('Docker image "delta-processor-axis:*" not exists on your machine\n'
                  'Please build the image first before trying this script.')
else:
    print('Docker image exists.')

Docker image exists.


### Run docker container

In [6]:
# Set volume of the container as temp directory, where the input file is located in.
volume_path = os.path.join(os.getcwd(), tmpdirname)

# Run docker container
client.containers.run(
    image='delta-processor-axis',
    auto_remove=True,
    volumes={volume_path: {'bind': '/data', 'mode': 'rw'}},
    command=f'-i {filename}'
)

b'%/var/lib/powercrust/powercrust -i horse.off\n'

### Check outputs

In [7]:
for file in os.listdir(tmpdirname):
    size = os.path.getsize(os.path.join(tmpdirname, file))
    size_mb = round(size / 1024 / 1024, 1)
    
    print(f'{file.ljust(25)}{size_mb}MB')

horse.off.pts            2.7MB
tempFile.txt             12.4MB
axis.off                 94.0MB
inpole                   3.4MB
pc.off                   74.0MB
horse.off                4.6MB
inpball                  4.5MB
outpole                  3.5MB
poleinfo                 99.4MB
axisface.off             39.3MB
